# RAG (Retrieval-Augmented Generation)

---

RAG en palabras simples es darle información al modelo para aumentar su ventana de contexto y que pueda responder preguntas especificas acerca de esos datos o información adicional.
[Aquí está la informacion sobre RAG](https://python.langchain.com/docs/use_cases/question_answering/).
Es recomendable que al comenzar con RAG se utilice LangChain, ya que es fácil de usar y bastante intuitivo.

¿Cómo le damos contexto a nuestro modelo? Bueno, primero utilizamos dos cosas, [Embeddings](https://developers.google.com/machine-learning/crash-course/embeddings/video-lecture?hl=es-419) y [VectorStores](https://python.langchain.com/docs/modules/data_connection/vectorstores/). **Leer esa documentación**.

Los pasos a seguir son los siguientes:

*   Cargo mis datos o documentos.
*   Parto estos documentos o datos en trozos.
*   Utilizo un modelo de Embeddings para crear Embeddings con mis documentos o datos.
*   Guardo estos Embeddings en una base de datos (VectorStore).
*   Cargo mi modelo.
*   Inicializo mi base de datos para que pueda servir para obtener información de ella
*   Indico que voy a utilizar esa base de datos para obtener la información.
*   Hago mi pregunta.
*   La pregunta pasa primero por la base de datos.
*   La base de datos recupera toda la informacion similar a la pregunta realizada.
*   Luego tanto el contexto como la pregunta es enviada al modelo para que haga la inferencia.

---

Recomiendo mucho leer la documentación sobre RAG de LangChain porque lo explica muy bien y tiene material visual con diagramas para entenderlo más fácil. También, para entender que es lo que pasa con tu información, te recomiendo leer la documentación sobre Embeddings y VectorStores.

# Requisitos
---
Para poder utilizar RAG en LangChain debes tener los siguientes paquetes de Python instalados.

*   `langchain`
*   `openai` (en el caso de que uses modelos como GPT)
*   `chromadb` (es la base de datos (VectorStore) donde se guardaran tus Embeddings)
*   `tiktoken`
*   `pypdf`

In [15]:
from langchain.llms import OpenAI
from langchain.document_loaders import WebBaseLoader
import os
import textwrap

os.environ["OPENAI_API_KEY"] = "..." # Aquí pon tu API key.


In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# Se carga el contenido de un PDF y se transforma a un archivo de texto.
# Aquí se utiliza un solo documento, pero se pueden usar más documentos.
loader = PyPDFLoader(
    "./docs/2018-DECRETO-REGLAMENTO-GENERAL-DE-DOCENCIA-DE-PREGRADO.pdf"
) # Fuente: https://docencia.udec.cl/documentos/reglamentos-de-pregrado/

# loader = UnstructuredCSVLoader("/content/Reporte_Sensores_Maquinas_Industriales.csv", mode="single")

# Esta línea carga los documentos como una variable para ser utilizada más adelante.
docs = loader.load()

print(docs)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

# Esta línea se encarga de partir el texto en trozos de 1200 Chars (caracteres) con un Overlap (superpisición) de 0.
# Lo ideal es tener un Overlap un poco más grande, pero hay que ir viendo cual sería el óptimo.
# La razón de partir el texto en trozos es principalmente facilitar el hacer Embeddings con el texto.
text_splitter = CharacterTextSplitter(chunk_size = 1200, chunk_overlap = 0)

# Aquí indicamos que queremos partir los documentos y guardarlos en una variable.
all_splits = text_splitter.split_documents(docs)

print(all_splits)

In [16]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# Aquí indicamos que queremos utilizar como modelo de Embedding text-embedding-ada-002,
# recomendado por openAI, pero puedes utilizar uno local (open source) de Huggingface.
EMBEDDING_MODEL = "text-embedding-ada-002" # Modelos de OpenAI y sus precios: https://openai.com/api/pricing/

# Aquí creamos un VestorStore, para almacenar los Embeddings en índices y facilitar la recuperación de los datos.
# Se indican los documentos que vas a pasar a Embeddings, aquí utilizo OpenAI (como modelo de Embedding),
# pero se puede cambiar, el nombre de la colección, no es relevante a menos que tengas varios VectorStores.
vectorstore = Chroma.from_documents(documents = all_splits, embedding = OpenAIEmbeddings(model = EMBEDDING_MODEL), collection_name = "data")

In [ ]:
## TEST

question = "Plan de Estudio"

# Aquí estoy testeando si me devuelve todo el texto que encuentre similar o que contenga "Plan de estudio".
docs = vectorstore.similarity_search(question)

# len(docs)

print(docs)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain import hub
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

# Aquí traemos un modelo de OpenAI, en este caso gpt-3.5-turbo. La temperatura indica la creatividad del modelo.
# Cuando quieres que te devuelve la información lo más parecida debes usar temperatura 0,
# para que sea más creativo se debe usar temperatura 1.
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)

# Aquí cargamos la Prompt que envuelve tanto el contexto como tu pregunta.
# Esto es para indicarle al modelo como debe actuar una vez recibe todo.
prompt = hub.pull("rlm/rag-prompt") # Fuente: https://smith.langchain.com/hub/rlm/rag-prompt

# Esto es para formatear los documentos.
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

# Aquí creamos la cadena de RAG, primero pasamos la base de datos como un Retriever, para obtener de ahí la información,
# luego indicamos como va a recibir la pregunta.
# Pasamos la Prompt, el modelo y un Parser (analizador) para la respuesta.
rag_chain = (
    {"context": vectorstore.as_retriever() | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("tu pregunta aquí") # reemplazar